# Task 4:
# News Topic Classifier
# Objective

The objective of this task was to build a text classification system that categorizes news headlines into one of four categories (World, Sports, Business, Sci/Tech). This was achieved by fine-tuning a pretrained BERT model (bert-base-uncased) on the AG News dataset, allowing the model to learn context-specific patterns in news language for accurate classification.

# Introduction

This task involved applying transfer learning with the BERT model, a widely used transformer-based architecture for natural language processing. The AG News dataset, containing 120,000 training and 7,600 testing samples across four balanced categories, was used. The main goal was to demonstrate how a large pretrained model can be fine-tuned for a specific task without training from scratch. The process included tokenizing the text using the BERT tokenizer, preparing the dataset for training, fine-tuning the classification head, and finally deploying the model for real-time interaction using Gradio.

# Overview

The workflow began with loading the AG News dataset from Hugging Face Datasets and splitting the training set into training and validation subsets. The bert-base-uncased tokenizer converted each news headline into token IDs and attention masks suitable for BERT input. The model was fine-tuned using Hugging Face’s Trainer API.

Evaluation was performed using accuracy and F1-score (weighted) to measure classification performance across all categories. The fine-tuned model and tokenizer were saved locally, and a Hugging Face pipeline was created for easy inference.

For deployment, a Gradio interface was implemented, allowing users to input custom headlines and instantly view the predicted category with scores.

# Summary

This task successfully delivered a fine-tuned BERT-based text classifier capable of categorizing news headlines into predefined topics with strong accuracy and F1 performance. It demonstrated key skills in natural language processing, including tokenization, transformer fine-tuning, and evaluation. Furthermore, lightweight deployment through Gradio showcased how machine learning models can be made interactive and user-friendly.

In [2]:
# Install required packages
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers==4.21.0 datasets accelerate gradio scikit-learn evaluate

# Import libraries
import torch
import numpy as np
import os
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
import evaluate
import gradio as gr

# Disable Weights & Biases logging (optional, prevents unnecessary logs)
os.environ["WANDB_DISABLED"] = "true"

# Check if GPU is available, otherwise fallback to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load the AG News dataset (train and test splits)
raw_ds = load_dataset("ag_news")

# Split training set into train (90%) and validation (10%)
splits = raw_ds["train"].train_test_split(test_size=0.1, seed=42)
train_ds = splits["train"]
val_ds = splits["test"]
test_ds = raw_ds["test"]

# Load tokenizer for BERT (converts text → tokens)
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to tokenize text data
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True)

# Apply tokenization to train/val/test datasets
train_tok = train_ds.map(tokenize, batched=True, remove_columns=["text"])
val_tok = val_ds.map(tokenize, batched=True, remove_columns=["text"])
test_tok = test_ds.map(tokenize, batched=True, remove_columns=["text"])

# Define label mapping (for readability of predictions)
id2label = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}
label2id = {v: k for k, v in id2label.items()}

# Load pretrained BERT model for classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=4,        # 4 categories in AG News dataset
    id2label=id2label,   # Map ID  label
    label2id=label2id    # Map label  ID
)

# Load metrics (accuracy and F1-score)
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

# Function to compute metrics during training/evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred                 # Predictions and true labels
    preds = np.argmax(logits, axis=-1)         # Convert logits predicted class
    return {
        "accuracy": acc_metric.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1_metric.compute(predictions=preds, references=labels, average="weighted")["f1"]
    }

# Data collator: automatically pads text to same length in each batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define training arguments
args = TrainingArguments(
    output_dir="bert-agnews",
    eval_strategy="epoch",       # use eval_strategy for older versions
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    seed=42,
    fp16=torch.cuda.is_available(),
    report_to=[]
)


# Create Hugging Face Trainer object
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train the model
print("Starting training")
trainer.train()
print("Training complete")

# Evaluate model on validation set
val_metrics = trainer.evaluate()
print("\nValidation Results:")
print(f"Accuracy: {val_metrics['eval_accuracy']:.4f}")
print(f"F1-Score: {val_metrics['eval_f1']:.4f}")

# Evaluate model on test set
test_metrics = trainer.evaluate(test_tok)
print("\nTest Results:")
print(f"Accuracy: {test_metrics['eval_accuracy']:.4f}")
print(f"F1-Score: {test_metrics['eval_f1']:.4f}")

# Save trained model and tokenizer to local directory
model.save_pretrained("./bert-news-classifier")
tokenizer.save_pretrained("./bert-news-classifier")
print("Model saved")

# Load trained model into a text classification pipeline
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="./bert-news-classifier",
    tokenizer="./bert-news-classifier",
    device=0 if torch.cuda.is_available() else -1  # Run on GPU if available
)

# Function to classify a news headline
def classify_news(text):
    if not text.strip():   # Handle empty input
        return "Enter a news headline"
    result = classifier(text)                # Run prediction
    confidence = result[0]['score']          # Confidence score
    category = result[0]['label']            # Predicted category
    return f"Category: {category}\nConfidence: {confidence:.2%}"

# Create Gradio app for interactive testing
demo = gr.Interface(
    fn=classify_news,
    inputs=gr.Textbox(label="News Headline", placeholder="Enter news headline"),
    outputs=gr.Textbox(label="Classification Result"),
    title="News Topic Classifier",
    examples = [
    "Microsoft announces new Windows update with security fixes",
    "Pakistan wins T20 cricket match against India",
    "SpaceX successfully lands reusable rocket",
    "Pakistan faces Flash Floods in August"
]
)
# Launch demo with public sharing
demo.launch(share=True)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)


ModuleNotFoundError: No module named 'evaluate'